<a href="https://colab.research.google.com/github/sajosa/DMA2023TeamD/blob/Sabines-branch/ReHealthCo_Rohdatenbank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReHealthCo Erstellen der Rohdatenbank 

## Vorbereitung 

###CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books (Ziel Reproduzierbarkeit)

In [ ]:
# Löschen aller Variablen
%reset -f

### Importieren der notwendigen Libraries: sqlite3, csv, pandas, ydata_profiling

In [ ]:
# import libraries
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
!pip install ydata_profiling
# from functools import reduce - benötigt?

In [ ]:
from ydata_profiling import ProfileReport

### mount drive to access data

In [ ]:
# mount drive to access data

In [ ]:
!git clone https://github.com/Fuenfgeld/DMA2023TeamD

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd /content/DMA2023TeamD

In [ ]:
# !ls

### create a database connection to SQLite database

In [ ]:
# Connect to source database
def create_connection(db_file):
  """ create a database connection to SQLite database"""
  conn = None
  try:
    conn = sqlite3.connect(db_file)
    return conn
  except Error as error:
    print("Error while connecting to sqlite", error)
  # finally:
  #   if conn:
  #     conn.close()

### Verbindung zur geteilten Ablage der Gruppe auf  google drive 

In [ ]:
material_path = "/content/DMA2023TeamD/"
# conn = create_connection() - verbindung zur Datenbank
# cur = conn.cursor() - objekt auf datenbank, das die SQL-Anfragen ausführen kann

## Erstellen der Rohdatenbank "rhc_database"
Ablauf:
1.   Erstellen der "noch" leeren Rohdatenbank "rhc_database" (Zieldatenbank)
2.   Erstellung der (leeren) Datenbanken der ausgewählten Synthia Module 

 a) asthma --> ast

 b) covid --> cov

 c) metabolic syndrom deseas --> met
 

### 1. Erstellen der "noch" leeren Rohdatenbank "rhc_database"

In [ ]:
# create a temporary empty rhc_database in the memory
rhc_db = f"{material_path}/source_dbs/rehealthco.db"
rhc_conn = create_connection(rhc_db)
rhc_cur = rhc_conn.cursor()
rhc_conn.commit()
print("Database connection is successfully")

### 2.a) Erstellung der (leeren) Datenbanken der ausgewählten Synthia Module asthma 

In [ ]:
# create a temporary empty ast_database in the memory
ast_db = f"{material_path}/source_dbs/asthma.db"
ast_conn = create_connection(ast_db)
ast_cur = ast_conn.cursor()
ast_conn.commit()
print("Database connection is successfully")

### 2.b)Erstellung der (leeren) Datenbanken der ausgewählten Synthia Module covid

In [ ]:
# create a temporary empty cov_database in the memory
cov_db = f"{material_path}/source_dbs/covid.db"
cov_conn = create_connection(cov_db)
cov_cur = cov_conn.cursor()
cov_conn.commit()
print("Database connection is successfully")

### 2.c) Erstellung der (leeren) Datenbanken der ausgewählten Synthia Module metabolic

In [ ]:
# create a temporary empty met_database in the memory
met_db = f"{material_path}/source_dbs/metabolic.db"
met_conn = create_connection(met_db)
met_cur = met_conn.cursor()
met_conn.commit()
print("Database connection is successfully")

## Erstellung der Tabellen in den erstellten DB (rhc, asthma, cov, met) mittels sql skript

In [ ]:
path = f"{material_path}/source_dbs/sql_create.txt"
myfile = ""
with open(path, "r") as f:
  myfile=f.read()
#  print(myfile)

In [ ]:
rhc_cur.executescript(myfile)
rhc_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in cov_database
rhc_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(rhc_cur.fetchall())

In [ ]:
ast_cur.executescript(myfile)
ast_conn.commit()
print("Successfully created tables in the database asthma.db")

In [ ]:
# List of tables in ast_database
ast_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(ast_cur.fetchall())

In [ ]:
cov_cur.executescript(myfile)
cov_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in cov_database
cov_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cov_cur.fetchall())

In [ ]:
met_cur.executescript(myfile)
met_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in met_database
met_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(met_cur.fetchall())

## Datenbanken (ast, cov, met) befüllen mit csv-Dateien

In [ ]:
csv_path_ast = f"{material_path}csv_data/asthma"
csv_path_cov = f"{material_path}csv_data/covid19"
csv_path_met = f"{material_path}csv_data/metabolic_syndrome_disease"

In [ ]:
# load the data into a Pandas DataFrame
astpat_df = pd.read_csv(f"{csv_path_ast}/patients.csv")
astcon_df = pd.read_csv(f"{csv_path_ast}/conditions.csv")
astmed_df = pd.read_csv(f"{csv_path_ast}/medications.csv")
astenc_df = pd.read_csv(f"{csv_path_ast}/encounters.csv")

covpat_df = pd.read_csv(f"{csv_path_cov}/patients.csv")
covcon_df = pd.read_csv(f"{csv_path_cov}/conditions.csv")
covmed_df = pd.read_csv(f"{csv_path_cov}/medications.csv")
covenc_df = pd.read_csv(f"{csv_path_cov}/encounters.csv")

metpat_df = pd.read_csv(f"{csv_path_met}/patients.csv")
metcon_df = pd.read_csv(f"{csv_path_met}/conditions.csv")
metmed_df = pd.read_csv(f"{csv_path_met}/medications.csv")
metenc_df = pd.read_csv(f"{csv_path_met}/encounters.csv")

# Abschluss
ast_conn.commit()
cov_conn.commit()
met_conn.commit()

In [ ]:
# Hinzufügen einer zusätzlichen Spalte mit dem Wert asthma/covid/metabolic; CAVE: späterer Copy-table Befehl ist kein df sondern ließt Ursprungstabelle, daher ggf. Tabellendatenbankart mit DEFAULT wert schon in txt-Datei einfügen
astpat_df['datenbankart'] = 'asthma'
astcon_df['datenbankart'] = 'asthma'
astmed_df['datenbankart'] = 'asthma'
astenc_df['datenbankart'] = 'asthma'

covpat_df['datenbankart'] = 'covid'
covcon_df['datenbankart'] = 'covid'
covmed_df['datenbankart'] = 'covid'
covenc_df['datenbankart'] = 'covid'

metpat_df['datenbankart'] = 'metabolic'
metcon_df['datenbankart'] = 'metabolic'
metmed_df['datenbankart'] = 'metabolic'
metenc_df['datenbankart'] = 'metabolic'

# Abschluss
ast_conn.commit()
cov_conn.commit()
met_conn.commit()

In [ ]:
# write the data to a sqlite table
astpat_df.to_sql("patients", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astcon_df.to_sql("conditions", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astmed_df.to_sql("medications", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astenc_df.to_sql("encounters", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

covpat_df.to_sql("patients", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covcon_df.to_sql("conditions", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covmed_df.to_sql("medications", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covenc_df.to_sql("encounters", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

metpat_df.to_sql("patients", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metcon_df.to_sql("conditions", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metmed_df.to_sql("medications", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metenc_df.to_sql("encounters", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

# Abschluss
ast_conn.commit()
cov_conn.commit()
met_conn.commit()

In [ ]:
# check dataframe asthma/covid19/metabolic
# print(astpat_df)
# print(astcon_df)
# print(astmed_df)
# print(astenc_df)

# print(covpat_df)
# print(covcon_df)
# print(covmed_df)
# print(covpro_df)

# print(metpat_df)
# print(metcon_df)
# print(metmed_df)
# print(metpro_df)

In [ ]:
# Check data in patients

# check data in a table astpat
# ast_cur.execute("SELECT * FROM patients;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covpat
# cov_cur.execute("SELECT * FROM patients;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metpat
# met_cur.execute("SELECT * FROM patients;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in conditions

# check data in a table astcon
# ast_cur.execute("SELECT * FROM conditions;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covcon
# cov_cur.execute("SELECT * FROM conditions;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metcon
# met_cur.execute("SELECT * FROM conditions;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in medications

# check data in a table astmed
# ast_cur.execute("SELECT * FROM medications;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covmed
# cov_cur.execute("SELECT * FROM medications;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metmed
# met_cur.execute("SELECT * FROM medications;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in encounters

# check data in a table astpro
# ast_cur.execute("SELECT * FROM encounters;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covpro
# cov_cur.execute("SELECT * FROM encounters;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metpro
# met_cur.execute("SELECT * FROM encounters;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

## Copy tables from ast, cov. met db to rhc database
  Patiens --> pat

  conditions --> con

  Medication --> met

  Encounters --> enc 


In [ ]:
# Get connections to the databases 
# von asthma nach rehealthco
def copy_table(rhc_conn, ast_cur, rhc_cur, table_name):
  ast_cur.execute(f'SELECT * FROM {table_name}')
  output = ast_cur.fetchall()   # Returns the results as a list.
  # Insert those contents into another table.
  for row in output:
      rowlen = len(row)
      repstring = ""
      for i in range(rowlen -1):
        repstring += "?,"
      repstring += "?"
      rhc_cur.execute(f'INSERT INTO {table_name} VALUES ({repstring})', row)
  # Cleanup
  rhc_conn.commit()

In [ ]:
# alle Tabellen vereinen in rehealthco
copy_table(rhc_conn, ast_cur, rhc_cur, "patients") # 1035 patienten
copy_table(rhc_conn, cov_cur, rhc_cur, "patients") # nun 2085 patienten
copy_table(rhc_conn, met_cur, rhc_cur, "patients") # nun 3113 patienten

copy_table(rhc_conn, ast_cur, rhc_cur, "conditions") # 260 conditions
copy_table(rhc_conn, cov_cur, rhc_cur, "conditions") # 6895 conditions
copy_table(rhc_conn, met_cur, rhc_cur, "conditions") # 13733 conditions

copy_table(rhc_conn, ast_cur, rhc_cur, "medications") # 1515 medications
copy_table(rhc_conn, cov_cur, rhc_cur, "medications") # 2910 medications
copy_table(rhc_conn, met_cur, rhc_cur, "medications") # 4625 medications

copy_table(rhc_conn, ast_cur, rhc_cur, "encounters") # 8522 encounters
copy_table(rhc_conn, cov_cur, rhc_cur, "encounters") # 35368 encounters
copy_table(rhc_conn, met_cur, rhc_cur, "encounters") # 63081 encounters


In [ ]:
# Tabelleninhalt rehealthco als pandas dataframe
rhcpat_df = pd.read_sql('''SELECT * FROM patients''', rhc_conn)
rhccon_df = pd.read_sql('''SELECT * FROM conditions''', rhc_conn)
rhcmed_df = pd.read_sql('''SELECT * FROM medications''', rhc_conn)
rhcenc_df = pd.read_sql('''SELECT * FROM encounters''', rhc_conn)

# print(rhcpat_df)
# print(rhccon_df)
# print(rhcmed_df)
# print(rhcenc_df)

Export csv file ReHealthCo Rohdatenbank

In [ ]:
rhcpat_df.to_csv('rhcpat_df', index=False)
rhccon_df.to_csv('rhccon_df', index=False)
rhcmed_df.to_csv('rhcmet_df', index=False)
rhcenc_df.to_csv('rhcenc_df', index=False)

Anzeigen Dateinamen in sample data

In [ ]:
! ls

# Bis hierhin final, danach noch experimentell

Index auf Patiens id

In [ ]:
# Tabelle patients
rhc_cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS IX_PATIENTS_Id ON patients (Id);")

In [ ]:
# show list indexes created for a table
rhc_cur.execute("PRAGMA index_list(patients)")
print(rhc_cur.fetchall())

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(0)

df = pd.DataFrame(np.random.randint(1,20,size=(20, 4)), columns=list('ABCD'))

print (df)

# To Do zur Finalisierung der Rohdatenbank

1. Index auf Patiens ID numeric als fortlaufende Nummer
2. Alter berechnen und in neuer Spalte (column) in patiens Tabelle einfügen

# Version Check

In [ ]:
#python Version
import sys
sys.version_info

In [ ]:
pd.__version__

In [ ]:
%pip install pandas

In [ ]:
sns.__version__

In [ ]:
matplotlib.__version__

In [ ]:
np.__version__

In [ ]:
sklearn.__version__

In [ ]:
graphviz.__version__

In [ ]:
%pip freeze